In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install --upgrade mxnet-cu100
!pip install autogluon

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pwd

/content


In [ ]:
!pip install split-folders

import splitfolders  # or import split_folders

input_folder = '/content/drive/MyDrive/Eye Diseases Virus Dateset Vol 1/JPG'

# Split with a ratio.
# To only split into training and validation set, set a tuple to `ratio`, i.e, `(.8, .2)`.
#Train, val, test
splitfolders.ratio(input_folder, output='/content/drive/MyDrive/Eye Diseases Virus Dateset Vol 1/ForAutogluon', 
                   seed=42, ratio=(.9, .0, .1), 
                   group_prefix=None) # default values


# Split val/test with a fixed number of items e.g. 100 for each set.
# To only split into training and validation set, use a single number to `fixed`, i.e., `10`.
# enable oversampling of imbalanced datasets, works only with fixed
splitfolders.fixed(input_folder, output='/content/drive/MyDrive/Eye Diseases Virus Dateset Vol 1/ForAutogluon', 
                   seed=42,fixed=(0, 83), oversample=False, group_prefix=None) 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Operation cancelled by user


ModuleNotFoundError: ignored

In [ ]:
from autogluon.vision import ImagePredictor, ImageDataset
dataset = ImageDataset.from_folder('/content/drive/MyDrive/Eye Diseases Virus Dateset Vol 1/ForAutogluon/train')

/usr/local/lib/python3.7/dist-packages/gluoncv/__init__.py:40: UserWarning: Both `mxnet==1.9.0` and `torch==1.12.1+cu113` are installed. You might encounter increased GPU memory footprint if both framework are used at the same time.
  warnings.warn(f'Both `mxnet=={mx.__version__}` and `torch=={torch.__version__}` are installed. '


In [ ]:
time_limit =  3 * 60 * 60 # 10mins
predictor = ImagePredictor()
predictor.fit(dataset, time_limit = time_limit)

INFO:autogluon.vision.predictor.predictor:Reset labels to [0, 1, 2, 3]
INFO:autogluon.vision.predictor.predictor:Randomly split train_data into train[946]/validation[106] splits.
INFO:autogluon.vision._gluoncv.image_classification:No GPU detected/allowed, using most conservative search space.
INFO:autogluon.vision._gluoncv.image_classification:Starting fit without HPO
INFO:TorchImageClassificationEstimator:modified configs(<old> != <new>): {
INFO:TorchImageClassificationEstimator:root.train.epochs    200 != 5
INFO:TorchImageClassificationEstimator:root.train.early_stop_patience -1 != 10
INFO:TorchImageClassificationEstimator:root.train.batch_size 32 != 16
INFO:TorchImageClassificationEstimator:root.train.early_stop_max_value 1.0 != inf
INFO:TorchImageClassificationEstimator:root.train.early_stop_baseline 0.0 != -inf
INFO:TorchImageClassificationEstimator:root.gpus            (0,) != ()
INFO:TorchImageClassificationEstimator:root.img_cls.model   resnet101 != resnet18
INFO:TorchImageClas

In [ ]:
print('Top-1 val acc: %.3f' % predictor.fit_summary()['valid_acc'])

Top-1 val acc: 0.689


In [ ]:
test_dataset = ImageDataset.from_folder('/content/drive/MyDrive/Eye Diseases Virus Dateset Vol 1/ForAutogluon/test')
pred = predictor.predict(test_dataset)
print(pred)

0      2
1      1
2      1
3      1
4      0
      ..
327    3
328    3
329    3
330    3
331    2
Name: label, Length: 332, dtype: int64


In [ ]:
test_dataset.head

<bound method NDFrame.head of                                                  image  label
0    /content/drive/MyDrive/Eye Diseases Virus Date...      0
1    /content/drive/MyDrive/Eye Diseases Virus Date...      0
2    /content/drive/MyDrive/Eye Diseases Virus Date...      0
3    /content/drive/MyDrive/Eye Diseases Virus Date...      0
4    /content/drive/MyDrive/Eye Diseases Virus Date...      0
..                                                 ...    ...
327  /content/drive/MyDrive/Eye Diseases Virus Date...      3
328  /content/drive/MyDrive/Eye Diseases Virus Date...      3
329  /content/drive/MyDrive/Eye Diseases Virus Date...      3
330  /content/drive/MyDrive/Eye Diseases Virus Date...      3
331  /content/drive/MyDrive/Eye Diseases Virus Date...      3

[332 rows x 2 columns]>

In [ ]:
score = predictor.evaluate(test_dataset)

INFO:TorchImageClassificationEstimator:[Epoch 2] validation: top1=0.599398 top5=1.000000


In [ ]:
print(score)

{'loss': 1.0248896170811481, 'top1': 0.5993975889251892, 'top5': 1.0}


In [ ]:
from sklearn.metrics import confusion_matrix
ans = test_dataset.label
print(ans)
cm = confusion_matrix(pred,ans)
print(cm)

0      0
1      0
2      0
3      0
4      0
      ..
327    3
328    3
329    3
330    3
331    3
Name: label, Length: 332, dtype: int64
[[17  8  7  0]
 [36 65 22  7]
 [20  8 47  6]
 [10  2  7 70]]
